In [2]:
import pyspark.sql
import pyspark
from pyspark.sql.functions import pandas_udf, PandasUDFType, col
from pyspark.sql.types import *
import pandas as pd
import numpy as np

session = pyspark.sql.SparkSession.builder \
    .master('spark://10.64.22.215:7077') \
    .appName('Vectorized_UDF_Example') \
    .config('spark.jars.packages','org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .config('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.sql.caseSensitive','true') \
    .config('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
    .config("spark.cores.max", "10") \
    .config('spark.executor.instances', '5') \
    .config('spark.executor.cores','2') \
    .config('spark.executor.memory','2g') \
    .getOrCreate()
    
sqlContext = session

print 'Spark version: ',sqlContext.version
print 'SparkSQL sesssion created'

df = sqlContext.sparkContext.parallelize([
    ("a", 0), ("a", 1), ("b", 30), ("b", -50)
]).toDF(["group", "power"])

pandas_df = pd.DataFrame(np.random.randn(5,3),columns=['col1','col2','col3'])
#convert to sql dataframe
spark_df = sqlContext.createDataFrame(pandas_df)
#convert to pandas dataframe
dff = spark_df.toPandas()

def below_threshold(threshold, group="group", power="power"):
    @pandas_udf("struct<group: string, below_threshold: boolean>",PandasUDFType.GROUPED_MAP)
    def below_threshold_(df):
        df = pd.DataFrame(
           df.groupby(group).apply(lambda x: (x[power] < threshold).any()))
        df.reset_index(inplace=True, drop=False)
        return df

    return below_threshold_


@pandas_udf("struct<group: string, below_threshold: boolean>",PandasUDFType.GROUPED_MAP)
def below_thresholdv2(df):
    group="group"
    power="power"
    df = pd.DataFrame(
        df.groupby(group).apply(lambda x: (x[power] < -40).any()))
    df.reset_index(inplace=True, drop=False)
    return df

# Use a decorator
@pandas_udf("struct<float>",PandasUDFType.SCALAR)
#input a,b are pandas series, the output is the result of two pandas series addition of same size
def func_udf(a,b):
    c = a + b
    return c

customSchema="struct<col1: double,col2: double,col3: double,col4: double>"
#=StructType(
#    [StructField("col1",DoubleType(),True)
#    ,StructField("col2",DoubleType(),True)
#    ,StructField("col3",DoubleType(),True)
#    ,StructField("col4",DoubleType(),True)]
#)

#GROUP_MAP type must take a single arg that is a pandas DataFrame.
#and retrun pandas dataframe
@pandas_udf(spark_df.schema,PandasUDFType.GROUPED_MAP)
def func_udfv2(df_):
    df_ = pd.DataFrame(
        df_.apply(lambda x: x.any())
    )
    df_.reset_index(inplace=True, drop=False)
    return df_

Spark version:  2.3.0
SparkSQL sesssion created


ParseException: u"\nextraneous input '<' expecting {'SELECT', 'FROM', 'ADD', 'AS', 'ALL', 'DISTINCT', 'WHERE', 'GROUP', 'BY', 'GROUPING', 'SETS', 'CUBE', 'ROLLUP', 'ORDER', 'HAVING', 'LIMIT', 'AT', 'OR', 'AND', 'IN', NOT, 'NO', 'EXISTS', 'BETWEEN', 'LIKE', RLIKE, 'IS', 'NULL', 'TRUE', 'FALSE', 'NULLS', 'ASC', 'DESC', 'FOR', 'INTERVAL', 'CASE', 'WHEN', 'THEN', 'ELSE', 'END', 'JOIN', 'CROSS', 'OUTER', 'INNER', 'LEFT', 'SEMI', 'RIGHT', 'FULL', 'NATURAL', 'ON', 'LATERAL', 'WINDOW', 'OVER', 'PARTITION', 'RANGE', 'ROWS', 'UNBOUNDED', 'PRECEDING', 'FOLLOWING', 'CURRENT', 'FIRST', 'AFTER', 'LAST', 'ROW', 'WITH', 'VALUES', 'CREATE', 'TABLE', 'DIRECTORY', 'VIEW', 'REPLACE', 'INSERT', 'DELETE', 'INTO', 'DESCRIBE', 'EXPLAIN', 'FORMAT', 'LOGICAL', 'CODEGEN', 'COST', 'CAST', 'SHOW', 'TABLES', 'COLUMNS', 'COLUMN', 'USE', 'PARTITIONS', 'FUNCTIONS', 'DROP', 'UNION', 'EXCEPT', 'MINUS', 'INTERSECT', 'TO', 'TABLESAMPLE', 'STRATIFY', 'ALTER', 'RENAME', 'ARRAY', 'MAP', 'STRUCT', 'COMMENT', 'SET', 'RESET', '

In [ ]:
df.groupBy("group").apply(below_threshold(-40)).show()

In [ ]:
#df.apply(below_thresholdv2(-40)).show()
df.groupBy("group").apply(below_thresholdv2).show()

In [ ]:
pdd=df.toPandas()

In [ ]:
pdd

In [ ]:
#func_udf = pandas_udf(func, FloatType())
spark_df.withColumn("c", func_udf(col("col2"), col("col3"))).show()

In [ ]:
spark_df.withColumn("c", func_udfv2).show()

In [ ]:
dff.apply(func_udfv2)